In [1]:
classpath.add("org.plotly-scala" %% "plotly-jupyter-scala" % "0.1.0")

Name: Compile Error
Message: <console>:18: error: not found: value classpath
       classpath.add("org.plotly-scala" %% "plotly-jupyter-scala" % "0.1.0")
       ^
<console>:18: error: value %% is not a member of String
       classpath.add("org.plotly-scala" %% "plotly-jupyter-scala" % "0.1.0")
                                        ^
StackTrace: 

In [6]:
def f(x :Int) = x*x

In [3]:
class square{
    def c = 2
}


In [4]:
val s = new square

In [2]:
import co.theasi.plotly
import util.Random

// Generate uniformly distributed x
val xs = (0 until 100)

// Generate random y
val ys = (0 until 100).map { i => i + 5.0 * Random.nextDouble }

val p = Plot().withScatter(xs, ys)

draw(p, "basic-scatter", writer.FileOptions(overwrite=true))
// returns  PlotFile(pbugnion:173,basic-scatter)

Name: Compile Error
Message: <console>:17: error: not found: value co
       import co.theasi.plotly
              ^
StackTrace: 

In [4]:
%%pyspark

1+1

2

In [3]:
import plotly._
import plotly.element._
                    
val trace1 = Scatter(
  Seq(1, 2, 3, 4),
  Seq(10, 15, 13, 17)
)

val trace2 = Scatter(
  Seq(1, 2, 3, 4),
  Seq(16, 5, 11, 9)
)

val data = Seq(trace1, trace2)

Plotly.plot("div-id", data)

Name: Compile Error
Message: <console>:17: error: not found: value plotly
       import plotly._
              ^
StackTrace: 

In [ ]:
%%pyspark
import plotly 
plotly.offline.init_notebook_mode() 
from plotly.graph_objs import *
from IPython.display import HTML, display
import numpy as np

s = np.linspace(0, 2 * np.pi, 240)
t = np.linspace(0, np.pi, 240)
tGrid, sGrid = np.meshgrid(s, t)

r = 2 + np.sin(7 * sGrid + 5 * tGrid)  # r = 2 + sin(7s+5t)
x = r * np.cos(sGrid) * np.sin(tGrid)  # x = r*cos(s)*sin(t)
y = r * np.sin(sGrid) * np.sin(tGrid)  # y = r*sin(s)*sin(t)
z = r * np.cos(tGrid)                  # z = r*cos(t)

surface = Surface(x=x, y=y, z=z)
data = Data([surface])

layout = Layout(
    title='Parametric Plot',
    scene=Scene(
        xaxis=XAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=YAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=ZAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = Figure(data=data, layout=layout) 

plot_html= plotly.offline.offline._plot_html(
        fig, show_link=False, link_text=" ", validate=True,
        default_width='100%', default_height=725, global_requirejs=True)[0].replace(
        "})});</script>",",\"displayModeBar\": false})});</script>") 
display(HTML(plot_html))


